
## graph based session intelligence

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("retailrocket/ecommerce-dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/nahomnadew/.cache/kagglehub/datasets/retailrocket/ecommerce-dataset/versions/2


In [1]:
#initial parameters

DATA_PATH = "data/kaggle/events.csv"  
SAMPLE_MAX_SESSIONS = 200000  
MIN_SESSION_LENGTH = 2  
NEO4J_URI = "bolt://localhost:7687"
NEO4J_AUTH = ("neo4j", "nahi1420")  
BATCH_SIZE = 5000  


In [2]:
#importing necessary libraries
import os
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from tqdm import tqdm
from neo4j import GraphDatabase
import networkx as nx
from sklearn.model_selection import train_test_split

print("pandas", pd.__version__)

pandas 2.3.3


In [3]:
# Load dataset (may be large)
assert os.path.exists(DATA_PATH), f"Data file not found: {DATA_PATH}"
print("Loading, this may take a while for the full RetailRocket dataset...")

# The RetailRocket 'events.csv' has columns similar to: sessionId, itemId, eventType, timestamp
usecols = None  
df = pd.read_csv(DATA_PATH, parse_dates=['timestamp'], low_memory=False)
print("Loaded rows:", len(df))
print("Columns:", df.columns.tolist())
df.head()

Loading, this may take a while for the full RetailRocket dataset...


/tmp/ipykernel_8259/3092328460.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(DATA_PATH, parse_dates=['timestamp'], low_memory=False)


Loaded rows: 2756101
Columns: ['timestamp', 'visitorid', 'event', 'itemid', 'transactionid']


,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [6]:
# Corrected RetailRocket column mapping
cols = [c.lower() for c in df.columns]
colmap = {}
if 'visitorid' in cols:
    colmap[[c for c in df.columns if c.lower()=='visitorid'][0]] = 'session_id'
if 'itemid' in cols:
    colmap[[c for c in df.columns if c.lower()=='itemid'][0]] = 'event_id'
if 'event' in cols:
    colmap[[c for c in df.columns if c.lower()=='event'][0]] = 'event_type'
if 'timestamp' in cols:
    colmap[[c for c in df.columns if c.lower()=='timestamp'][0]] = 'timestamp'

df = df.rename(columns=colmap)
print('Renamed columns mapping:', colmap)

assert 'session_id' in df.columns and 'event_id' in df.columns and 'timestamp' in df.columns

# Add step_index per session (order by timestamp)
df = df.sort_values(['session_id','timestamp'])
df['step_index'] = df.groupby('session_id').cumcount() + 1

# Filter out sessions that are too short
session_lengths = df.groupby('session_id').size()
valid_sessions = session_lengths[session_lengths >= MIN_SESSION_LENGTH].index
df = df[df['session_id'].isin(valid_sessions)].copy()
print("After filtering short sessions:", df['session_id'].nunique(), "unique sessions,", len(df), "rows")

Renamed columns mapping: {'visitorid': 'session_id', 'event': 'event_type', 'timestamp': 'timestamp'}
After filtering short sessions: 406020 unique sessions, 1754541 rows


In [7]:
# pick the first N sessions (keeps temporal order inside sessions)
if SAMPLE_MAX_SESSIONS is not None:
    unique_sess = df['session_id'].drop_duplicates().iloc[:SAMPLE_MAX_SESSIONS]
    df = df[df['session_id'].isin(unique_sess)].copy()
    print("Sampled sessions:", df['session_id'].nunique(), "rows:", len(df))

Sampled sessions: 200000 rows: 869349


In [9]:
# Save a sampled CSV for quick checks 
sample_out = "data/sampled_sessions.csv"
df.to_csv(sample_out, index=False)
print("Saved sample to", sample_out)

Saved sample to data/sampled_sessions.csv
